Animesh Guchhait

MDS202004

In [ ]:
pip install gensim==4.1.2

     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
# Important libraries
import math
import numpy as np
import pandas as pd
import random

import json
import nltk
nltk.download('stopwords')
from nltk . corpus import stopwords
import numpy as np
from copy import deepcopy
import re
import sklearn
from scipy import linalg
from scipy import sparse
import os
from operator import itemgetter
from random import shuffle
from tqdm import tqdm
from collections import defaultdict
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from gensim.models import word2vec
from gensim.models import Word2Vec
import logging
from sklearn.metrics.pairwise import cosine_similarity
 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


I have already preprocessed the given data in assignment 1, but there I have used diffrent type of lemmatization and stemming in preprocessing and that destorted many words. So here I don't want to use that preprocessed data. So in next few steps I have preprocessed the json files like assignment 1. But here I am not using diffrent type of stemming or lemmatization, just using a single one and rest of the preprocessing is same as assignment 1.

In [ ]:

def extract_text(filename):
    '''
    Extract the text from the file name (json file) and
    index the content from paper_id, title, abstract and body_text fields
    Retuns - text of title, abstract and bodt_text
    '''

    file = open(filename)
    body_text = ""
    abstract = ""
    title = ""
    paper_id = ""

    paper_content = json.load(file)

    #get the paper_id
    if 'paper_id' in paper_content:
        paper_id = paper_content['paper_id']
        
    #get the title, if available
    if 'title' in paper_content['metadata']:
        title = paper_content['metadata']['title']
    #get abstract.text, if available
    if 'abstract' in paper_content:
        for abs in paper_content['abstract']:
            abstract = abstract + abs['text']
    if 'body_text' in paper_content:
        for bt in paper_content['body_text']:
            body_text = body_text + bt['text']


   
    return (title + ' ' + abstract + ' ' + body_text + ' ').lower()



In [ ]:
#split the text and removes non alphaneumeric characters

def split_and_re_alphaneumeric(words):
  words=words.split()
  for i in range(len(words)):
    words[i]=re.sub('[\W\_]','',words[i])
    
  return words

In [ ]:
# Remove stop words, '' and neumeric
def remove_sw(words):
    stop_words = set(stopwords.words('english'))
    return [i for i in words if i not in stop_words and i not in [''] and  i.isalpha()]

In [ ]:
# to lemmatize each words.
def  lemmatization(words):
  lemmatizer = WordNetLemmatizer()
 
  for i in range(len(words)):
    w=words[i]
    words[i]=lemmatizer.lemmatize(w)
  return words


In [ ]:
''' Here using this file_formating function , by giving a folder location and json filenames present in 
that folder, we can get extracted files(using extract_text function in this function) as elements of a list
i.e. the first element of the list will be text file which contain the text of the first json file name (like that)'''


def file_formatting(filenames,path):
    fnames =deepcopy(filenames)
    files = []
    c=0
    for filename in fnames:
        #if c>= 3000:
         # break
        try:
            file=extract_text(path+'/'+filename)
            files.append(file)
           # c=c+1
        except:
            ind = filenames.index(filename)
            #print('hoche na')
            filenames.pop(ind)
    return files

In [ ]:
!cp /content/drive/MyDrive/NLP/pdf_json.zip pdf_json.zip  # Unzip the provided zip data in the colab disk 
!unzip -oq pdf_json.zip
print('Unzip Completed')

Unzip Completed


In [ ]:
# getting path and file name of our pdf_json folder where we unzip our given data
path='/content/pdf_json'
filenames = os.listdir(path)

In [ ]:
files=file_formatting(filenames,path)  # Called the file-formating function to get extracted text documents as elements of a list

In [ ]:
len(files)

56528

In [ ]:
'''All the preprocessing task are done here in steps to all the text files
   step:
   1.  split and re_alphaneumeric : to split the text and removes non alphaneumeric characters
   2. remove_sw :  to remove stop words, '' and neumeric
   3. word lemmatization
   4. then save it to another list and delete the previous unpreprocessed file '''


file_store=[]
for i in tqdm(range(len(files))):            
  words=split_and_re_alphaneumeric(files[0])
  words=remove_sw(words)
  words=lemmatization(words)
  file_store.append(words)
  del words
  del files[0]


100%|██████████| 56528/56528 [15:51<00:00, 59.43it/s]


In [ ]:
del files

In [ ]:
# By this code I have write the preprocessed documents in text files and have zip those files and saved that for future use.

os.mkdir('/content/preproc1/')
for i in range(len(file_store)):
  with open('/content/preproc1/'+filenames[i][:-5]+'.txt','w') as f:
    f.write(' ' .join(file_store[i]))
!zip -r preproc_2.zip /content/preproc1/
!cp preproc_2.zip /content/drive/MyDrive/NLP

Streaming output truncated to the last 5000 lines.
  adding: content/preproc1/5b54bc83e427dd28addc54ee29db2fd529b79e2b.txt (deflated 64%)
  adding: content/preproc1/15ace9a200a05c72d12721a8448c6959809e135e.txt (deflated 55%)
  adding: content/preproc1/05fcde0b042a42310a19070fae6de9a2a9935e71.txt (deflated 64%)
  adding: content/preproc1/53350e9c4aca69dcb9ff9c5339ada938a9066ef7.txt (deflated 71%)
  adding: content/preproc1/6b4b32c9c9f84ee5d0a74196f5d9378a955adac1.txt (deflated 68%)
  adding: content/preproc1/2f6052a0d41ac2f8a5bb92444cf020a7f260ba49.txt (deflated 68%)
  adding: content/preproc1/59cf76e798b85f37841bc1fd7e181f9351144082.txt (deflated 68%)
  adding: content/preproc1/6152353a5dc0e674b07967985f45498260ef2cda.txt (deflated 66%)
  adding: content/preproc1/226e654c8633175ebbec1e348db4b155169ce111.txt (deflated 61%)
  adding: content/preproc1/15f2b1915443fff0466f0dd89c7c0ab6761833b4.txt (deflated 69%)
  adding: content/preproc1/4cdd787c243c8962f2fa44ed603071b2d53fb67f.txt (deflat

This link is for the preprocesed documents :

https://drive.google.com/file/d/1irn5oETFHlvCvKeaDGjWkLeSTzJ3yHPM/view?usp=sharing

I have already saved the preprocessed files for fututre use. Now for the next part of this assignment, I have unzip the preprocessed file here in colab disk in the very next step and have performed remaining part of the assignment.

In [ ]:
!cp /content/drive/MyDrive/NLP/preproc_2.zip preproc_2.zip
!unzip -q preproc_2.zip
print('Unzip Completed')

Unzip Completed


In [ ]:
 # getting path and file name of our preprocessed files 
path='/content/content/preproc1'
filenames = os.listdir(path)

In [ ]:
'''Given the filenames, this function will go through each file and 
   1. will count total number of words/token and will store it in count
   2. will count the frequency of each word/token and will create a matrix for that named d
   3. will also count total number of vocabulary for each 10,000 words count done and  will store that on list named l '''

def frequency_matrix(filenames):
  d = defaultdict(int)
  count=0  
  l=[]
  for i in tqdm(range(len(filenames))):
    with open('/content/content/preproc1/'+filenames[i],'r') as f:    # Reading each file
      t=f.read().split()
    for j in t:
      d[j]=d[j]+1
      count=count+1
      if count % 10000 ==0:                               # For each 10,000 words, count total number of vocabulary
        l.append([len(d.keys()),count])
    del t

  #for i in d.keys():
    #d[i]=d[i]/count
  d=dict(d)
  d=pd.DataFrame(d.values(),d.keys())
  d.columns=['frequency']
  d=d.sort_values("frequency", axis = 0, ascending = False)        # sorting the word frequency matrix according to the frequency 
  return(d,count,l)


In [ ]:
d,count,l=frequency_matrix(filenames)

100%|██████████| 56528/56528 [00:58<00:00, 960.90it/s]


In [ ]:
d[:14000]

,frequency
patient,883123
cell,741056
study,652674
et,608947
al,599496
...,...
advisor,591
benthamiana,591
rapidity,591
openly,591


In [ ]:
d_freq=d[:14000].to_dict()['frequency']


In [ ]:
d_freq # dictionary containing keys as vocabulary and values as frequency

{'patient': 883123,
 'cell': 741056,
 'study': 652674,
 'et': 608947,
 'al': 599496,
 'virus': 545893,
 'infection': 504753,
 'also': 476844,
 'may': 449935,
 'disease': 444577,
 'case': 403353,
 'protein': 398796,
 'data': 389438,
 'using': 357684,
 'result': 351374,
 'used': 346721,
 'time': 312498,
 'model': 305778,
 'one': 303040,
 'group': 301965,
 'health': 285851,
 'level': 279956,
 'number': 278536,
 'viral': 274887,
 'two': 261799,
 'de': 259980,
 'however': 251638,
 'analysis': 249284,
 'day': 249228,
 'human': 249148,
 'p': 247548,
 'system': 245309,
 'effect': 242988,
 'treatment': 239977,
 'response': 233373,
 'clinical': 231956,
 'different': 227523,
 'risk': 227148,
 'use': 221096,
 'fig': 219587,
 'rate': 214465,
 'sample': 210601,
 'control': 210227,
 'could': 200271,
 'high': 199187,
 'figure': 198102,
 'reported': 197320,
 'method': 194327,
 'gene': 194270,
 'first': 193161,
 'respiratory': 191347,
 'n': 189252,
 'found': 187828,
 'factor': 182919,
 'activity': 18176

In [ ]:
l=d[:14000].index
d_list = defaultdict(int)
for i in range(len(l)):
  d_list[l[i]]=i

In [ ]:
d_list  # dictionary containing keys as vocabulary and values as index

defaultdict(int,
            {'patient': 0,
             'cell': 1,
             'study': 2,
             'et': 3,
             'al': 4,
             'virus': 5,
             'infection': 6,
             'also': 7,
             'may': 8,
             'disease': 9,
             'case': 10,
             'protein': 11,
             'data': 12,
             'using': 13,
             'result': 14,
             'used': 15,
             'time': 16,
             'model': 17,
             'one': 18,
             'group': 19,
             'health': 20,
             'level': 21,
             'number': 22,
             'viral': 23,
             'two': 24,
             'de': 25,
             'however': 26,
             'analysis': 27,
             'day': 28,
             'human': 29,
             'p': 30,
             'system': 31,
             'effect': 32,
             'treatment': 33,
             'response': 34,
             'clinical': 35,
             'different': 36,
             'risk': 37,

In [ ]:
del d,count, l  # To recover space

In [ ]:
''' To create coal matrix 
    have taken window size of 4'''


arr=np.zeros((len(d_list.keys()),len(d_list.keys())))  # initialization
for i in tqdm(range(len(filenames))):
  with open('/content/content/preproc1/'+filenames[i],'r') as f:
      token=f.read().split()
      for i in range(len(token)):
        if token[i] in d_list.keys():
          m=d_list[token[i]]

          if len(token)-i-1<4:           # window size =4
              k=4
              for t in range(i+1,len(token)):
                if token[t] in d_list.keys():
                  n=d_list[token[t]]

                  arr[m][n]=arr[m][n]+k
                  arr[n][m]=arr[n][m]+k
                k=k-1
          else:
              k=4
              for t in range(i+1,i+5):
                if token[t] in d_list.keys():
                  n=d_list[token[t]]
                  arr[m][n]=arr[m][n]+k
                  arr[n][m]=arr[n][m]+k
                k=k-1

100%|██████████| 56528/56528 [20:24<00:00, 46.17it/s]


In [ ]:
df=pd.DataFrame(arr)  # transforming to dataframe

In [ ]:
path = '/content/drive/MyDrive/NLP/coal.csv'      # saving coal matrix to drive for future use 

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)

The saved coal matrix is :

https://drive.google.com/file/d/1-1zm487ZCpeygxKF8WJP2wot3AjqPJUo/view?usp=sharing

Now I am going to use this saved coal matrix for finding correlation matrix and SVD.

In [ ]:
!cp /content/drive/MyDrive/NLP/coal.csv coal.csv
df=pd.read_csv('/content/coal.csv')              # Read coal matrix

In [ ]:
df.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '13990', '13991', '13992', '13993', '13994', '13995', '13996', '13997',
       '13998', '13999'],
      dtype='object', length=14001)

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
arr=np.array(df)

In [ ]:
arr   # coal matrix

array([[2.83078e+05, 3.05960e+04, 1.41134e+05, ..., 6.20000e+01,
        3.70000e+01, 4.40000e+01],
       [3.05960e+04, 3.76736e+05, 3.53310e+04, ..., 2.70000e+01,
        0.00000e+00, 2.00000e+01],
       [1.41134e+05, 3.53310e+04, 1.10272e+05, ..., 2.80000e+01,
        5.00000e+01, 5.20000e+01],
       ...,
       [6.20000e+01, 2.70000e+01, 2.80000e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.70000e+01, 0.00000e+00, 5.00000e+01, ..., 0.00000e+00,
        0.00000e+00, 2.00000e+00],
       [4.40000e+01, 2.00000e+01, 5.20000e+01, ..., 0.00000e+00,
        2.00000e+00, 6.00000e+00]])

In [ ]:
del df

In [ ]:
''' To create correlation matrix '''

def corel(arr):
    t=arr.sum()
    l=sum(arr)
    arr1=np.zeros((len(arr),len(arr)))
    for i in tqdm(range(len(arr))):
        for j in range(i,len(arr)):
          
            w_a_b=arr[i][j]
            w_a_j=l[j]
            w_i_b=l[i]
            
            w_crr=(t*w_a_b-w_a_j*w_i_b)/(w_a_j*w_i_b*(t-w_i_b)*(t-w_a_j))**0.5   #  correlation formula
            arr1[i][j]=w_crr
            arr1[j][i]=w_crr
    return arr1

In [ ]:
df2=corel(arr)

100%|██████████| 14000/14000 [03:25<00:00, 68.27it/s]  


In [ ]:
df2     #correlation matrix

array([[ 9.65089433e-03, -5.00720978e-03,  3.46273855e-03, ...,
        -5.53339038e-05, -1.06422703e-04, -9.62011573e-05],
       [-5.00720978e-03,  2.15466917e-02, -3.27628630e-03, ...,
        -1.13367270e-04, -1.78352282e-04, -1.30275404e-04],
       [ 3.46273855e-03, -3.27628630e-03,  3.51230909e-03, ...,
        -9.56849545e-05, -2.67173225e-05, -2.80687318e-05],
       ...,
       [-5.53339038e-05, -1.13367270e-04, -9.56849545e-05, ...,
        -5.22209280e-06, -5.08074402e-06, -5.19488976e-06],
       [-1.06422703e-04, -1.78352282e-04, -2.67173225e-05, ...,
        -5.08074402e-06, -4.94322119e-06,  1.81664386e-04],
       [-9.62011573e-05, -1.30275404e-04, -2.80687318e-05, ...,
        -5.19488976e-06,  1.81664386e-04,  5.42680145e-04]])

In [ ]:
del arr

In [ ]:
''' here for the correlation matrix, if we have negative value,we will replace that with zero 
and for positive value , we will replace with square root of that value '''

def replace_values(df2):
  arr1=np.zeros((len(df2),len(df2)))
  for i in tqdm(range(len(df2))):
    for j in range(len(df2)):
        if df2[i][j]<0:
            arr1[i][j]=0
        else:
            arr1[i][j]=np.sqrt(df2[i][j])
  return arr1 

In [ ]:
df3=replace_values(df2)

100%|██████████| 14000/14000 [03:21<00:00, 69.51it/s]


In [ ]:
df3  # now it is much more sparse 

array([[0.09823897, 0.        , 0.05884504, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.14678791, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05884504, 0.        , 0.05926474, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.01347829],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01347829,
        0.0232955 ]])

In [ ]:
del df2

In [ ]:
# Appling SVD on that df3 sparse matrix
m, n = df3.shape
U, s, Vh = sparse.linalg.svds(df3,k=50)
U.shape,  s.shape, Vh.shape

((14000, 50), (50,), (50, 14000))

In [ ]:
# save U, s and Vh in drive

from numpy import save
save('/content/drive/MyDrive/NLP/U.npy', U)
save('/content/drive/MyDrive/NLP/Vh.npy', Vh)
save('/content/drive/MyDrive/NLP/s.npy', s)

Link for U, s and Vh are given accordingly:

U : https://drive.google.com/file/d/1nd_mFanFTUCROShB2LqghbiktdjcgrL4/view?usp=sharing

s : https://drive.google.com/file/d/1-4Bd8sn0R5giAfpxRZ4tOykFWZ9aULzk/view?usp=sharing

Vh : https://drive.google.com/file/d/1-65xFFkiUQJBlzAWi2eddV9hi8KUh32M/view?usp=sharing

In [ ]:
# Loading U, Vh,s  from drive
U=np.load('/content/drive/MyDrive/NLP/U.npy')
Vh=np.load('/content/drive/MyDrive/NLP/Vh.npy')
s=np.load('/content/drive/MyDrive/NLP/s.npy')

In [ ]:
U=U.astype(np.float32)  # changing data type
s=s.astype(np.float32)
s=np.diag(s)   # to create diagonal matrix from s


In [ ]:
s.shape

(50, 50)

In [ ]:
# svd_vec= U * s

svd_vec=U@s

In [ ]:
''' I am adding up all the text file in a single text file and 
saving it with the name covid19_corpus.txt to use it in gensim word2vec model '''


path='/content/content/preproc1'
filenames = os.listdir(path)

def text(filenames,path):
  with open('covid19_corpus.txt','a') as f:
    for x in filenames:
      with open('/content/content/preproc1/'+x,'r') as f1:
        t=f1.read()
        f.write(t)
  print('ready')



In [ ]:
text(filenames,path)

ready


In [ ]:
''' here in this function I am giving input of d_freq (dictionary for vocabulary with frequency)  and svd_vec
    it will take by default text file as covid19_corpus.txt
    window size =4
    epochs=10       
    
    return 
    it return the trained model which will be saved in drive'''

def compute_word_vectors(d_freq,svd_vec): 
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    text = word2vec.Text8Corpus('covid19_corpus.txt')
    model = word2vec.Word2Vec( vector_size=50, window=4, min_count=50, workers=4, epochs=10)
    model.build_vocab_from_freq(d_freq)
    model.wv.vectors = svd_vec
    count=0
    for i in text:     # to count total number of words
      for j in i: 
        count=count+1
    #print(count)
    model.train(text, total_words=count , epochs= model.epochs)
    model.save('/content/drive/MyDrive/NLP/word2vec.model.bin')


In [ ]:
compute_word_vectors(d_freq,svd_vec)

2022-05-17 15:20:42,081 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=50, alpha=0.025)', 'datetime': '2022-05-17T15:20:42.081195', 'gensim': '4.1.2', 'python': '3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2022-05-17 15:20:42,091 : INFO : Processing provided word frequencies
2022-05-17 15:20:42,093 : INFO : collected 14000 unique word types, with total frequency of 117172251
2022-05-17 15:20:42,095 : INFO : Creating a fresh vocabulary
2022-05-17 15:20:42,180 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=50 retains 14000 unique words (100.0%% of original 14000, drops 0)', 'datetime': '2022-05-17T15:20:42.180586', 'gensim': '4.1.2', 'python': '3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2022-05-17 15:20:42,182 : INFO : Word2Vec lifecycle event {'msg': 

The saved model is in the following link:

https://drive.google.com/file/d/1eJtqWdJyk594OCHdXhF_lzBpBVrWhGeN/view?usp=sharing

It can be used in future.

In [ ]:
model_word2vec = Word2Vec.load('/content/drive/MyDrive/NLP/word2vec.model.bin')   # load the word2vec model

2022-05-17 15:51:53,374 : INFO : loading Word2Vec object from /content/drive/MyDrive/NLP/word2vec.model.bin
2022-05-17 15:51:53,400 : INFO : loading wv recursively from /content/drive/MyDrive/NLP/word2vec.model.bin.wv.* with mmap=None
2022-05-17 15:51:53,402 : INFO : setting ignored attribute cum_table to None
2022-05-17 15:51:53,572 : INFO : Word2Vec lifecycle event {'fname': '/content/drive/MyDrive/NLP/word2vec.model.bin', 'datetime': '2022-05-17T15:51:53.572783', 'gensim': '4.1.2', 'python': '3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


In [ ]:
t=np.array([model_word2vec.wv['use']]) 

In [ ]:
t  # it shows word2vec of the word 'use'

array([[ 5.4541826 ,  3.83442   ,  0.92745227, -1.5616142 ,  0.79119015,
         1.7350136 ,  2.4440987 , -0.5818897 , -0.4193181 , -0.7483776 ,
        -0.9702736 ,  1.001613  , -2.552457  ,  2.4897518 ,  4.2041492 ,
         0.269991  ,  1.3528161 ,  2.5786257 ,  0.19657244, -0.28129318,
         0.09414467,  2.254788  , -0.13232836,  2.1807165 ,  3.461481  ,
        -1.992047  ,  1.2541376 ,  1.9305576 ,  0.43082777, -0.9966315 ,
         0.6899271 , -1.1810462 ,  3.4314833 ,  0.63708794,  2.2301564 ,
         3.2782393 , -0.09120215,  3.2097387 ,  1.6914824 , -1.0246164 ,
        -2.7121043 , -3.7632213 ,  3.9951973 ,  0.08489602,  3.1067336 ,
        -1.4227644 , -1.5156115 , -0.13359901, -0.09152357,  0.8726839 ]],
      dtype=float32)

In [ ]:
d_list.keys()  # Vocabulary list

dict_keys(['patient', 'cell', 'study', 'et', 'al', 'virus', 'infection', 'also', 'may', 'disease', 'case', 'protein', 'data', 'using', 'result', 'used', 'time', 'model', 'one', 'group', 'health', 'level', 'number', 'viral', 'two', 'de', 'however', 'analysis', 'day', 'human', 'p', 'system', 'effect', 'treatment', 'response', 'clinical', 'different', 'risk', 'use', 'fig', 'rate', 'sample', 'control', 'could', 'high', 'figure', 'reported', 'method', 'gene', 'first', 'respiratory', 'n', 'found', 'factor', 'activity', 'table', 'associated', 'including', 'c', 'population', 'well', 'test', 'year', 'q', 'r', 'individual', 'expression', 'value', 'antibody', 'change', 'compared', 'due', 'type', 'care', 'new', 'based', 'shown', 'mouse', 'higher', 'increased', 'rna', 'le', 'significant', 'infected', 'e', 'increase', 'within', 'sequence', 'severe', 'b', 'pandemic', 'important', 'many', 'positive', 'among', 'observed', 'three', 'role', 'function', 'age', 'information', 'present', 'potential', 'showe

In [ ]:
my_list_of_words=['patient','disease', 'virus','treatment','pandemic','influenza','symptom','lung','chronic','healthcare']  # my list of words to find similarity

In [ ]:
''' using this function I am going to find 5 most similar words to 
 each of my list of words
 
 it is basically try to find cosine similarity between two words. so, for a given word, it will 
 find vector form of the word from word2vec model and for the available other words in our vocabullary 
 it will do the same from word2vec model. And then it will check top 5 most similar words to the given word. '''


def similarity_matrix(l,d_list,model_word2vec):
  for i in  l:
    l1=[[-99,None],[-99,None],[-99,None],[-99,None],[-99,None]]
    for j in d_list.keys():
      if i == j:
        continue
      similarity=cosine_similarity(np.array([model_word2vec.wv[i]]),np.array([model_word2vec.wv[j]]))
      if similarity > l1[0][0]:
        l1[0]=[similarity,j]
        l1.sort()
    l1.sort()
    print('Similar word to',i)
    print()
    l1.reverse()
    print(pd.DataFrame(l1,columns=['similarity','word']))
    print()


In [ ]:
similarity_matrix(my_list_of_words,d_list,model_word2vec)

Similar word to patient

      similarity          word
0  [[0.8055382]]   patientsthe
1  [[0.7492214]]    patientsin
2  [[0.7480515]]  hospitalized
3  [[0.7304004]]  hospitalised
4  [[0.6962173]]     inpatient

Similar word to disease

       similarity        word
0   [[0.8717566]]  diseasethe
1  [[0.69555753]]     illness
2   [[0.6726584]]   infection
3   [[0.6507666]]     ailment
4    [[0.631497]]   pathology

Similar word to virus

       similarity           word
0  [[0.79007703]]  paramyxovirus
1   [[0.7820976]]     poliovirus
2  [[0.77409244]]       reovirus
3  [[0.74248457]]          viral
4   [[0.7375012]]       virusthe

Similar word to treatment

       similarity          word
0   [[0.8825907]]       therapy
1   [[0.7797239]]   prophylaxis
2  [[0.77757806]]  chemotherapy
3   [[0.7653471]]   monotherapy
4   [[0.7599099]]       regimen

Similar word to pandemic

       similarity           word
0   [[0.8242407]]    pandemicthe
1   [[0.8117206]]         crisis
2  [[0.78692716